# Notebook 3: Baysian Models

### Machine Learning Basic Module
Florian Walter, Tobias Jülg, Pierre Krack

### General Information About Implementation Assignments

**Note:** From this week on, we will also provide exercises that need to be completed in the Jupyter Notebook itself. Thus, you are allowed to modify the notebook at the indicated sections.

We will use the Jupyter Notebook for our implementation exercises. The task description will be provided in the notebook. The code is also run in the notebook. However, the implementation itself is done in additional files which are imported into the notebook. The contents of a Python file could, for example, look similar to what is shown below:
```python
def f():
    ########################################################################
    # YOUR CODE
    # TODO: Implement this function
    ########################################################################
    pass
    ########################################################################
    # END OF YOUR CODE
    ########################################################################
```
To complete the exercise, remove the `pass` command and use space only inside the `YOUR CODE` block to provide a solution. No other lines within the file should be changed to ensure a valid submission.

### Imports

In [ ]:
%reload_ext autoreload
%autoreload 2

from functools import partial
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from bayes_regression import likelihood, posterior_distribution, predictive_distribution
from naive_bayes import BernoulliNaiveBayes, bernoulli_naive_bayes_scikit

## Naive Bayes

### Exercise 3.1 Naive Bayes from Scratch
The Naive Bayes Classifier is a probabilistic classifier that makes classifications using Bayes' theorem. It's "naive" because it assumes that each feature is independent of the others when given the class variable. One of its strengths is its simplicity and its ability to handle an extremely large number of features. Its use cases include text classification, sentiment analysis, and spam filtering.

In this exercise, we will implement a Naive Bayes Classifier with Bernoulli likelihoods from scratch. Bernoulli likelihoods are used when our features are binary (0 or 1). We will use a small synthetic dataset to derive all necessary steps by hand before implementing them in Python.
The features of the dataset ($x_i$) describe whether a user likes the song $i$ or not. We know from a certain number of users (our training set) whether they also liked the target song. We would like to calculate the probability that a new user likes the target song given their preferences for the songs $x_i$. Our dataset, with 4 training samples and one test sample, is given below:

|             | ID | $x_1$ | $x_2$ | $x_3$ | Whether the user likes the target song |
|:------------|:---|:---|:---|:---|:---------------------------------------|
| Training data | 1  | 0  | 1  | 1  | Y                                      |
|     -"-          | 2  | 0  | 0  | 1  | N                                      |
|      -"-         | 3  | 0  | 0  | 0  | Y                                      |
|         -"-      | 4  | 1  | 1  | 0  | Y                                      |
| Testing case   | 5  | 1  | 1  | 0  | ?                                      |


In Naive Bayes we would like to get the probability that a sample with  $d$ features $x = (x_1, ..., x_d)$ is in class $k\in\{1, ..., K\}$, also called the posterior probability $p(y_k|x)$. Applying Bayes' theorem leads to
$$\underbrace{p(y_k|x)}_{\text{posterior}} = \frac{ \overbrace{p(x|y_k)}^{\text{likelihood}} \overbrace{p(y_k)}^{\text{prior}}}{\underbrace{p(x)}_\text{evidence}}$$


- **Prior probability $p(y_k)$**: This reflects how classes are generally distributed without considering any specific features. It's the baseline chance of each class occurring, which can be set uniformly or determined from data.
- **Posterior probability $p(y_k | x)$**: This is updated from the prior probability when we factor in the observed data, giving us the probability of a class given the features observed.
- **Likelihood $p(x | y_k) = p(x_1, ..., x_d|y_k)$**: The likelihood is the joint probability of observing a set of features in a given class. It becomes complex with an increase in features, but Naive Bayes simplifies this by assuming independence of the features in $x$.

>**Question 3.1.1:** Given this independence assumption, how can $p(x | y_k)$ be rewritten with its feature probabilities $p(x_i|y_k)$?
>
> Give the answer in the cell below:



$$
p(x | y_k) = p(x_1, ..., x_d|y_k) = 
$$






- The **evidence** $p(x)$ is based only on the general distribution of features and is not tied to specific classes. Consequently, it acts as a normalization constant. This implies that the posterior is in proportion to the prior and likelihood:

$$p(y_k|x) \propto p(x|y_k)\cdot p(y_k)$$




Since we know that a probability distribution always has to sum up to one:
$$\sum_{k\in K} p(y_k|x) = 1$$
we can use the proportion from above and calculate the normalization factor of the evidence with it to finally get our wanted posterior $p(y_k|x)$ probability.

Now let's calculate these steps for the song data given above. First we calculate the prior $p(Y)$ and $p(N)$. Since we are dealing with binary variables we can use the MLE of the Bernoulli distribution as an estimator for these probabilities (or in other words: we use the *Laplace Probability*):
$$p(Y) = \frac{\#Y}{\#Y + \#N} = \frac{3}{4}$$
$$p(N) = \frac{\#N}{\#Y + \#N} = \frac{1}{4}$$

Since we have to classed $y_k$ (N and Y) we have to calculate two posteriors $p(N|x)$ and p(Y|x) where $x=(1, 1, 0)$ is given from our test sample for which we want to calculate the posterior.

$$p(N|x) \propto p(N) \cdot p(x_1=1|N)\cdot p(x_2=1|N)\cdot p(x_3=0|N)$$
$p(Y|x)$ is analoge to that.


In order to calculate the likelihoods we can again apply the Laplace Rule of Succession onto our training data. However, this time, we will also use [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) with a smoothing factor of one. The smoothing factor avoids cases where no event has been observed but we still do not want to put zero probability in our estimator as the zero would propagate through our equations. For example $p(x_1=1|N)$ has no samples and thus would be assigned zero probability without Laplace smoothing. With Laplace smoothing we get:

$$p(x_1=1|N) = \frac{\#(x_1=1 \text{ and } N) + 1}{\#N + 2} = \frac{0 + 1}{1 + 2} = \frac{1}{3}$$
$$p(x_1=1|Y) = \frac{1 + 1}{3 + 2} = \frac{2}{5}$$
and so on.

After calculating all likelihoods, we get
$$\frac{p(N|x)}{p(Y|x)}\propto \frac{125}{1458}$$
and using the fact that probabilities need to sum up to one ($p(N|x) + p(Y|x) = 1$) we get $p(Y|x) = 92.1\%$ and $p(N|x) = 7.9\%$.


> **Task 3.1:** Now we had a look at the theory and an example, let's use the dataset from above and implement the Naive Bayes for Bernoulli distributions in Python.
> To do so implement the following functions in file `naive_bayes.py` (preferably in the order given below):
> - `_label_indices`
> - `_priors`
> - `_likelihood`
> - `_posteriors`
> - `fit`
> - `predict`
> 
> Use the docstrings to get more information and hints about the functions. The cell below defines the data set in python code and runs the functions you have to implement. If you run the cell, you should get the following output which is the same as we calculated by hand above:
> 
> ```python
> (['Y'], [{'Y': 0.9210360075805433, 'N': 0.07896399241945673}])
> ```

In [ ]:
X_train = np.array([
    [0, 1, 1],
    [0, 0, 1],
    [0, 0, 0],
    [1, 1, 0]
])
Y_train = ["Y", "N", "Y", "Y"]
X_test = np.array([[1, 1, 0]])
bnb = BernoulliNaiveBayes()
bnb.fit(X_train, Y_train)
bnb.predict(X_test)

> **Question 3.1.2:** What sort of estimator from the ones that you learned in the course does Naive Bayes classifier correspond to? Justify your answer.
>
>Provide your answer in the cell below.

> **Question 3.1.3:** Is Naive Bayes a discriminative or a generative probabilistic model? Justify your answer.
>
> Provide your answer in the cell below.

### Exercise 3.2 Naive Bayes with Scikit-Learn
> Task 3.2 Now that we have implemented Naive Bayes from scratch, we will also implement it using the `BernoulliNB` class from Scikit learn.
> Please complete the function `bernoulli_naive_bayes_scikit` in [naive_bayes.py](naive_bayes.py). The input data has the same format as in the exercise before.
> Furthermore, please return the same tuple as in the `predict` function in your previous implementation (list of predicted labels, list of dictionary containing the probabilities for each label).
> 
> When you are done, come back and check the cell below. It should return `True` if both of your implementations return the same results, which they should.

In [ ]:
def equal(a, b):
    return a[0] == b[0] and np.all([[np.allclose(a[key], b[key]) for key in a.keys()]for a, b in zip(a[1], b[1])])

equal(bernoulli_naive_bayes_scikit(X_train, Y_train, X_test), bnb.predict(X_test))

## Bayesian regression
In this exercise, you will implement Bayesian regression from scratch.
Bayesian regression, like other regression methods, attempts to associate an input value or vector, the independent variable in Bayesian terminology, with a continuous output value or vector, the dependent variable.
The approach is somewhat different but, as you will see, yields similar results.

We use the notation from C. Bishop (c.f. the "Mathematical notation" section at the beggining of the book for more details) and implement Bayesian regression using the equations from chapter 3.3.

|||
|-----------|---------|
|$\mathbf{X}$| The indepent values of our dataset.|
|$\mathbf{t}$| The targets corresponding to the $X$ values, i.e. the dependent variables (dependent on $X$).|
|$t$         | A single element inside $\mathbf{t}$|
|$N$         | The number of elements in our dataset (number of values in $X$ and $\mathbf{t}$)|
|$D$         | The dimensionality of the independent variables in $X$|


### Random data
Bayesian regression starts with the assumption that the data comes from a distribution. We will start by generating some random data.
For the entire exercise, we will take the noise precision, $\beta$ as given.
Precision is just the inverse of variance and the noise precision represents the measurement accuracy.
The assumption is that the data was measured somehow, and we know the precision of our measurement instruments.

In [ ]:
N = 20 # Size of the dataset
D = 1 # Dimensionality of the independent variable
beta = 1 # Measurement precision we take as given
np.random.seed(42)
X = np.linspace(-1, 1, N) + np.concatenate((np.repeat(-.8, int(N/2)), np.repeat(.8, int((N+1)/2))))
X = X.reshape(N, D)
# We use the multivariate normal distribution from scipy making our solution general
target_distribution = multivariate_normal(mean = np.sin(2 * X).flatten(), cov = np.eye(N) * 1/beta)
# Sample the targets from the distribution
t = target_distribution.rvs().reshape(N, D)
plt.scatter(X, t)
plt.show()

### Exercise 3.3 Bayesian regression from scratch
Now, as it has been the case in the coding assignments before this one, we want to find $w$ automatically.
Bayesian regression uses an approach that is different from least squares regression: Instead of calculating an error and minimizing it, we start with a prior distribution.
The prior $p(w)$ is a distribution over the weights of our model.

Using evidence in the form a dataset of independent-dependent variable pairs (pairs taken from $X$ and $\mathbf{t}$),
we can compute the posterior distribution $p(w|t)$, which gives us a distribution over the weights of the model, incorporating both the prior and the evidence.

This posterior distribution is computed using Bayes theorem:
$$\underbrace{P(w|\mathbf{t})}_{\text{posterior}} = \frac{\overbrace{P(\mathbf{t}|w)}^{\text{likelihood}}\overbrace{P(w)}^{\text{prior}}}{\underbrace{P(\mathbf{t})}_{\text{evidence}}}$$

### Likelihood
We already have the prior, next we compute the likelihood function $p(\mathbf{t} | w)$: the probability of seeing the targets $\mathbf{t}$ given the model's weight $w$.

The formula for the likelihood is the following (Bishop 3.10 p. 141):
$$p(\mathbf{t}|\mathbf{X}, \mathbf{w}, \beta) = \prod_{n=1}^N \mathcal{N}(t_n|\mathbf{w}^T \phi(\mathbf{x}_n), \beta^{-1})$$
where $\mathbf{t}$ is a column vector containing all targets, $X$ is a column vector of the corresponding independent variables, $w$ are the model's weights and $\beta$ is the precision parameter.
Remember that we assume that the targets $\mathbf{t}$ come from a normal distribution.
For each individual target vector $t_n$ (in our case this is a vector of size one) in computes the probability of seeing that target vector, given the current model.

> **Task 3.3.1** open [`bayesian_regression.py`](./bayesian_regression.py) and implement the likelihood function. Use scipy's [`multivariate_normal`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multivariate_normal.html) function for the multivariate gaussian distribution.
The computed likelihood should be very small since you are multiplying $N=20$ values that are less than one with each other.

In [ ]:
def polynomial_basis(x, degree):
    return np.array([np.power(x[0], i) for i in reversed(range(degree))])
degree = 6
phi = partial(polynomial_basis, degree=degree)
likelihood(t=t,
           X=X,
           w = np.zeros(degree),
           beta = 1,
           phi = phi
          )

### The evidence and the posterior

The next step is to compute the evidence $p(\mathbf{t}) = \int p(\mathbf{t} | w)p(w) \text{d}w$
Unfortunately, as you can see, this involves an integration step, which means we have to either analytically integrate for every different dataset $\mathbf{t}$ and different weights $w$ or settle for an approximation using numerical integration methods.

Except that we don't have to: using the equation 2.116 in Bishop, we can skip the complicated integration and evaluate the posterior direclty as $p(w|t) = \mathcal{N}(w|m_n, S_n)$ with 
$$m_N = S_N(S_0^{-1}m_0+\beta\Phi^Tt)$$
$$S_N^{-1} = S_0^{-1} \beta \Phi^T\Phi$$

We further simplify the computation by assuming that our prior is a zero-mean isotropic Gaussian ("isotropic" = "identical in all directions" i.e. the covariance matrix is a diagonal matrix $\sigma^2I$), parameterized only through a single precision parameter $\alpha$:
$$p(w|\alpha) = \mathcal{N}(w|0, \alpha^{-1} I)$$

> **Question 3.3.1** Is it reasonable to restrict ourselves to zero vector for the prior's weight parameters? Hint: Can you think of a desirable property for the final weights of a model?
>
>Please provide your answer in the cell below.

The corresponding posterior distribution is then given by $p(w|t) = \mathcal{N}(w|m_n, S_n)$ with
$$m_N = \beta S_N \Phi^Tt$$
$$S_N^{-1} = \alpha I + \beta \Phi^T\Phi$$
>**Task 3.3.2** open [`bayesian_regression.py`](bayesian_regression.py) and complete the `posterior_distribution()` function.

Now run the next function which plots the model (a polynomial basis function) using the most likely weights (i.e. the means since the [mode](https://en.wikipedia.org/wiki/Mode_(statistics)) of a gaussian is equal to its mean).
Because the the mean of a Gaussian coincides with its mode, what we do here is the same as maximum a posteriori estimation.

In [ ]:
posterior = posterior_distribution(X, t, 1, beta, phi)
xvals = np.linspace(-2, 2, 100)
Phi = np.array([phi(np.array(x,ndmin=1)) for x in xvals])
y = posterior.mean @ Phi.T
fig = plt.figure()
ax = fig.add_subplot()
ax.scatter(X, t)
ax.plot(xvals, y)
ax.set_ylim(-5, 5)
plt.show()

### Predictive distribution
Now we have the posterior distribution over our weights.
Because in our case this a Gaussian distribution, we can immediately get the most likely weights since they are equal to the mean.
However in general, this is not the case: any skewed distribution will have a mode different to its mean.

Bayesian regression has one more trick up its sleeve. We can compute the so-called "predictive distribution" $p(t|\mathbf{t}, \alpha, \beta)$, which gives us a probability distribution for a single target $t$ corresponding to a single input $x$. The advantage here, is that we get a free uncertainty measure for our model: the standard deviation.

The formula for the predictive distribution is given by Bishop 3.3.2 equations 3.58 and 3.59:
$$p(t|x,\mathbf{t},\alpha, \beta) = \mathcal{N}(t|m_N^T\phi(x), \sigma_N^2(x))$$
where the variance $\sigma_N^2(x)$ is given by
$$\sigma_N^2(x) = \frac{1}{\beta} + \phi(x)^TS_N\phi(x)$$

> **Task 3.3.3** Open [`bayes_regression.py`](./bayes_regression.py) and implement the function `predictive_distribution`. Then run the next cell to see your model's predictions along with the standard deviations that represent the model's uncertainty.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.scatter(X, t, label="targets")
means = []
devs = []
alpha = 0.01

for x in xvals:
    x = np.array(x, ndmin=1)
    d = predictive_distribution(x, X, t, alpha, beta, phi)
    means.append(d.mean)
    devs.append(d.cov)
means = np.array(means).flatten()
devs = np.array(devs).flatten()
ax.plot(xvals, means, label="prediction")
ax.fill_between(xvals, means - devs, means + devs, color="tab:blue", alpha=0.3, label="standard deviation")
ax.set_ylim(-8, 8)
ax.legend()
plt.show()